In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from nltk.stem.porter import *
stemmer = PorterStemmer()
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
import nltk
import gensim
from gensim import corpora
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/matt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv('/Users/matt/Downloads/liar_dataset/train.csv',sep=',')

In [3]:
data['the label'].unique()

array(['FALSE', 'half-true', 'mostly-true', 'TRUE', 'barely-true',
       'pants-fire'], dtype=object)

In [4]:
data.describe()

,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts
count,10263.000000,10263.000000,10263.000000,10263.000000,10263.000000
mean,11.549839,13.304394,17.168762,16.465166,6.202670
std,18.992224,24.132578,35.899983,36.202646,16.121555
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,3.000000,3.000000,1.000000
75%,12.000000,15.000000,13.000000,11.000000,5.000000
max,70.000000,114.000000,160.000000,163.000000,105.000000


In [5]:
data.head()

,the ID of the statement ([ID].json),the label,the statement,the subject,the speaker,the speaker's job title,the state info,the party affiliation,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue
0,2635.json,FALSE,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,FALSE,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [6]:
X = data.iloc[:, 2:].values
y = data.loc[:, ['the label']].values  

In [7]:
data.astype(str)

,the ID of the statement ([ID].json),the label,the statement,the subject,the speaker,the speaker's job title,the state info,the party affiliation,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue
0,2635.json,FALSE,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,FALSE,Health care reform legislation is likely to ma...,health-care,blog-posting,nan,nan,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,nan,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
5,12465.json,TRUE,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
6,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,nan,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.
7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa."
8,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,nan,nan,organization,0.0,0.0,1.0,0.0,1.0,a website
9,9741.json,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video


In [8]:
data.dropna(inplace=True)

In [9]:
special_char_list = [':', ';', '?', '}', ')', '{', '(', '$','%','...',':']
for special_char in special_char_list:
    data=data.replace(special_char, '')
    print(data)


      the ID of the statement ([ID].json)    the label  \
0                               2635.json        FALSE   
1                              10540.json    half-true   
2                                324.json  mostly-true   
5                              12465.json         TRUE   
7                                153.json    half-true   
9                               9741.json  mostly-true   
10                              7115.json  mostly-true   
11                              4148.json    half-true   
12                              5947.json        FALSE   
14                              8705.json  barely-true   
15                             10683.json    half-true   
16                               620.json         TRUE   
18                             12372.json    half-true   
19                             12385.json  mostly-true   
20                             10173.json        FALSE   
22                             12408.json  mostly-true   
24            

      the ID of the statement ([ID].json)    the label  \
0                               2635.json        FALSE   
1                              10540.json    half-true   
2                                324.json  mostly-true   
5                              12465.json         TRUE   
7                                153.json    half-true   
9                               9741.json  mostly-true   
10                              7115.json  mostly-true   
11                              4148.json    half-true   
12                              5947.json        FALSE   
14                              8705.json  barely-true   
15                             10683.json    half-true   
16                               620.json         TRUE   
18                             12372.json    half-true   
19                             12385.json  mostly-true   
20                             10173.json        FALSE   
22                             12408.json  mostly-true   
24            

      the ID of the statement ([ID].json)    the label  \
0                               2635.json        FALSE   
1                              10540.json    half-true   
2                                324.json  mostly-true   
5                              12465.json         TRUE   
7                                153.json    half-true   
9                               9741.json  mostly-true   
10                              7115.json  mostly-true   
11                              4148.json    half-true   
12                              5947.json        FALSE   
14                              8705.json  barely-true   
15                             10683.json    half-true   
16                               620.json         TRUE   
18                             12372.json    half-true   
19                             12385.json  mostly-true   
20                             10173.json        FALSE   
22                             12408.json  mostly-true   
24            

In [10]:
data = data.astype(str).apply(lambda x: x.str.lower())
data = data.astype(str).apply(lambda x: x.str.decode('utf8'))

In [11]:
data

,the ID of the statement ([ID].json),the label,the statement,the subject,the speaker,the speaker's job title,the state info,the party affiliation,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue
0,2635.json,false,says the annies list political group supports ...,abortion,dwayne-bohac,state representative,texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,when did the decline of coal start? it started...,"energy,history,job-accomplishments",scott-surovell,state delegate,virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"hillary clinton agrees with john mccain ""by vo...",foreign-policy,barack-obama,president,illinois,democrat,70.0,71.0,160.0,163.0,9.0,denver
5,12465.json,true,the chicago bears have had more starting quart...,education,robin-vos,wisconsin assembly speaker,wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece
7,153.json,half-true,i'm the only person on this stage who has work...,ethics,barack-obama,president,illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a democratic debate in philadelphia, pa."
9,9741.json,mostly-true,says gop primary opponents glenn grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,state representative,wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video
10,7115.json,mostly-true,"for the first time in history, the share of th...",elections,robert-menendez,u.s. senator,new jersey,democrat,1.0,3.0,1.0,3.0,0.0,a speech
11,4148.json,half-true,"since 2000, nearly 12 million americans have s...","economy,jobs,new-hampshire-2012,poverty",bernie-s,u.s. senator,vermont,independent,18.0,12.0,22.0,41.0,0.0,a tweet
12,5947.json,false,when mitt romney was governor of massachusetts...,"history,state-budget",mitt-romney,former governor,massachusetts,republican,34.0,32.0,58.0,33.0,19.0,an interview with cbn news
14,8705.json,barely-true,most of the (affordable care act) has already ...,health-care,george-will,columnist,maryland,columnist,7.0,6.0,3.0,5.0,1.0,"comments on ""fox news sunday"""


In [12]:
data.dropna(inplace=True)

In [13]:
data.drop(columns=['the ID of the statement ([ID].json)'],inplace=True)

In [14]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 

In [15]:
lemma = WordNetLemmatizer()

In [16]:
for w_index in data['the statement']:
    print(nltk.word_tokenize(w_index))

[u'says', u'the', u'annies', u'list', u'political', u'group', u'supports', u'third-trimester', u'abortions', u'on', u'demand', u'.']
[u'when', u'did', u'the', u'decline', u'of', u'coal', u'start', u'?', u'it', u'started', u'when', u'natural', u'gas', u'took', u'off', u'that', u'started', u'to', u'begin', u'in', u'(', u'president', u'george', u'w.', u')', u'bushs', u'administration', u'.']
[u'hillary', u'clinton', u'agrees', u'with', u'john', u'mccain', u'``', u'by', u'voting', u'to', u'give', u'george', u'bush', u'the', u'benefit', u'of', u'the', u'doubt', u'on', u'iran', u'.', u"''"]
[u'the', u'chicago', u'bears', u'have', u'had', u'more', u'starting', u'quarterbacks', u'in', u'the', u'last', u'10', u'years', u'than', u'the', u'total', u'number', u'of', u'tenured', u'(', u'uw', u')', u'faculty', u'fired', u'during', u'the', u'last', u'two', u'decades', u'.']
[u'i', u"'m", u'the', u'only', u'person', u'on', u'this', u'stage', u'who', u'has', u'worked', u'actively', u'just', u'last', u'

[u'harry', u'reid', u'has', u'voted', u'to', u'give', u'special', u'tax', u'breaks', u'to', u'illegal', u'aliens', u'.']
[u'job', u'growth', u'in', u'the', u'united', u'states', u'is', u'now', u'at', u'the', u'fastest', u'pace', u'in', u'this', u'countrys', u'history', u'.']
[u'on', u'how', u'to', u'respond', u'to', u'puerto', u'ricos', u'debt', u'crisis', u'.']
[u'the', u'state', u'of', u'rhode', u'island', u'has', u'the', u'worst', u'state-funded', u'pension', u'in', u'the', u'country', u'.']
[u'more', u'than', u'any', u'republican', u'governor', u'in', u'the', u'2014', u'elections', u',', u'i', u'took', u'a', u'higher', u'percentage', u'of', u'the', u'republican', u'voteand', u'yet', u'i', u'also', u'carried', u'independents', u'by', u'12', u'points', u'.']
[u'after', u'the', u'2012', u'redistricting', u',', u'wisconsin', u'has', u'competitive', u'congressional', u'districts', u'.']
[u'i', u"'ve", u'cast', u'a', u'couple', u'of', u'99-1', u'votes', u'and', u'been', u'the', u'lone', 

[u'when', u'the', u'salmonella', u'source', u'was', u'finally', u'identified', u',', u'fda', u'officials', u'had', u'to', u'wait', u'for', u'industry', u'approval', u'before', u'they', u'could', u'go', u'live', u'with', u'the', u'[', u'peanut', u']', u'recall', u'.']
[u'women', u'are', u'paid', u'77', u'cents', u'for', u'every', u'$', u'1', u'men', u'get', u'for', u'the', u'same', u'work', u'.']
[u'as', u'governor', u',', u'mitt', u'romney', u'raised', u'nursing', u'home', u'fees', u'eight', u'times', u'.']
[u'says', u'if', u'texas', u'abortion', u'measure', u'passes', u',', u'someone', u'living', u'in', u'el', u'paso', u'would', u'have', u'to', u'drive', u'550', u'miles', u'each', u'way', u'to', u'san', u'antonio', u'for', u'something', u'as', u'simple', u'as', u'cervical', u'cancer', u'screening', u'.']
[u'says', u'that', u'wall', u'street', u'hedge', u'fund', u'managers', u'pay', u'a', u'lower', u'tax', u'rate', u'than', u'does', u'a', u'sheet', u'metal', u'worker', u'in', u'parma',

[u'says', u'the', u'obama', u'administration', u'never', u'responded', u'to', u'his', u'2012', u'letter', u'flagging', u'the', u'uptick', u'in', u'unaccompanied', u'children', u'crossing', u'the', u'u.s.-mexico', u'border', u'.']
[u'president', u'obama', u'has', u'spent', u'over', u'$', u'2', u'million', u'in', u'legal', u'fees', u'defending', u'lawsuits', u'about', u'his', u'birth', u'certificate', u'.']
[u'says', u'this', u'house', u'processed', u'and', u'approved', u'as', u'many', u'bills', u'as', u'previous', u'legislatures', u'.']
[u'biden', u'was', u'responsible', u'for', u'``', u'ending', u'genocide', u'in', u'bosnia', u'.', u"''"]
[u'the', u'last', u'three', u'days', u'of', u'early', u'voting', u'happen', u'to', u'be', u'the', u'most', u'populated', u'days', u'of', u'voting', u'that', u'we', u'have', u'in', u'early', u'voting', u'.']
[u'repealing', u'the', u'sales', u'tax', u'on', u'boats', u'in', u'rhode', u'island', u'has', u'spawned', u'2,000', u'companies', u',', u'7,000', 

[u'says', u'state', u'senate', u'leaders', u'have', u'accepted', u'$', u'142,400', u'in', u'campaign', u'contributions', u'from', u'dentists', u'and', u'pharmacists', u'during', u'the', u'past', u'two', u'years', u'.']
[u'says', u'bill', u'white', u'is', u'refusing', u'to', u'debate', u'.']
[u'some', u'republicans', u'who', u'voted', u'against', u'the', u'stimulus', u'``', u'have', u'been', u'all', u'too', u'happy', u'to', u'claim', u'credit', u'for', u'recovery', u'act', u'projects', u'and', u'the', u'jobs', u'those', u'projects', u'have', u'produced', u'.', u'they', u'come', u'to', u'the', u'ribbon', u'cuttings', u'.', u"''"]
[u'theres', u'nothing', u'in', u'the', u'[', u'rhodemap', u'ri', u']', u'plan', u'that', u'takes', u'away', u'local', u'government', u'control', u'and', u'local', u'zoning', u'control', u'.']
[u'more', u'than', u'80', u'percent', u'of', u'georgians', u'dont', u'support', u'legislation', u'to', u'ease', u'gun', u'restrictions', u'in', u'houses', u'of', u'worship'

...

[u'says', u'taxes', u'were', u'raised', u'115', u'times', u'in', u'the', u'eight', u'years', u'before', u'i', u'became', u'governor', u'and', u'those', u'increases', u'were', u'part', u'of', u'a', u'path', u'that', u'led', u'to', u'wealth', u'and', u'jobs', u'and', u'people', u'leaving', u'our', u'state', u'.']
[u'the', u'georgia', u'secretary', u'of', u'states', u'office', u'and', u'the', u'georgia', u'state', u'elections', u'board', u'investigate', u'and', u'penalize', u'hundreds', u'of', u'people', u'guilty', u'of', u'election', u'fraud', u'.']
[u'the', u'obama', u'administration', u'``', u'has', u'been', u'constrained', u'by', u'the', u'oil', u'pollution', u'act', u'of', u'1990', u',', u'which', u'basically', u'gives', u'the', u'responsible', u'party', u'the', u'lead', u'role', u'in', u'trying', u'to', u'not', u'only', u'fix', u'the', u'problem', u',', u'but', u'contain', u'the', u'problem', u'.', u"''"]
[u'electric', u'utility', u'firstenergys', u'annual', u'revenues', u'are', u'g

[u'donald', u'trumps', u'strategy', u'is', u'pretty', u'simple', u'.', u'they', u'have', u'even', u'said', u'in', u'his', u'campaign', u'its', u'to', u'get', u'women', u'to', u'stay', u'home', u',', u'get', u'young', u'people', u'to', u'stay', u'home', u',', u'get', u'people', u'of', u'color', u'to', u'stay', u'home', u',', u'and', u'get', u'a', u'lot', u'smart', u',', u'intelligent', u'men', u'to', u'stay', u'home', u',', u'too', u'.']
[u'the', u'u.s.', u'has', u'a', u'record', u'number', u'of', u'oil', u'rigs', u'operating', u'right', u'now', u'more', u'working', u'oil', u'and', u'gas', u'rigs', u'than', u'the', u'rest', u'of', u'the', u'world', u'combined', u'.']
[u'ive', u'cut', u'taxes', u'for', u'...', u'middle-class', u'families', u',', u'small', u'businesses', u'.']
[u'the', u'gun', u'epidemic', u'is', u'the', u'leading', u'cause', u'of', u'death', u'of', u'young', u'african-american', u'men', u',', u'more', u'than', u'the', u'next', u'nine', u'causes', u'put', u'together', u'.

[u'we', u'have', u'invested', u'record', u'funding', u'in', u'protecting', u'our', u'environment', u'.']
[u'$', u'2.4', u'million', u'of', u'the', u'nih', u'dollars', u'was', u'spent', u'on', u'origami', u'condoms', u'.']
[u'half', u'of', u'illegal', u'immigrants', u'come', u'on', u'legal', u'visas', u'and', u'then', u'overstay', u'.']
[u'the', u'alternative', u'minimum', u'tax', u'was', u'never', u'intended', u'to', u'hit', u'people', u'in', u'middle', u'income', u',', u'upper', u'middle', u'income', u'.', u'it', u'was', u'meant', u'for', u'people', u'who', u'are', u'rich', u'and', u'evading', u'taxes', u'.']
[u'aids', u'was', u'transmitted', u'to', u'humans', u'because', u'one', u'guy', u'had', u'sex', u'with', u'a', u'monkey', u'and', u'then', u'started', u'having', u'sex', u'with', u'men', u'.']
[u'says', u'6', u'percent', u'more', u'republicans', u'across', u'the', u'state', u'voted', u'than', u'democrats', u'.']
[u'blocking', u'travel', u'from', u'countries', u'with', u'ebola', u

[u'the', u'book', u'little', u'red', u'riding', u'hood', u'is', u'something', u'thats', u'been', u'banned', u'in', u'america', u',', u'but', u'not', u'assault', u'weapons', u'.']
[u'the', u'nations', u'food', u'supply', u'is', u'99.99', u'percent', u'safe', u'.']
[u'in', u'rhode', u'island', u'today', u',', u'25', u'percent', u'of', u'our', u'households', u'either', u'dont', u'have', u'a', u'bank', u'account', u'at', u'all', u',', u'or', u'they', u'have', u'a', u'bank', u'account', u'and', u'theyre', u'still', u'relying', u'on', u'high-cost', u'financial', u'services', u'like', u'payday', u'loans', u',', u'pawn', u'shop', u'check', u'cashing', u'and', u'so', u'on', u'.']
[u'our', u'whole', u'food', u'system', u'in', u'this', u'country', u'...', u'most', u'of', u'it', u'is', u'genetically', u'engineered', u'.', u'and', u'(', u'there', u'is', u')', u'no', u'testing', u'for', u'the', u'health', u'consequences', u'of', u'this', u'food', u'.']
[u'weve', u'saved', u'over', u'$', u'100', u'mi

[u'the', u'truth', u'is', u'david', u'cicilline', u'voted', u'for', u'tougher', u'laws', u'against', u'child', u'abusers', u',', u'domestic', u'abusers', u'and', u'abusers', u'of', u'the', u'elderly', u'.']
[u'weve', u'been', u'able', u'to', u'create', u'about', u'60,000', u'net', u'new', u'jobs', u'in', u'virginia', u'since', u'february', u'.']
[u'if', u'you', u'have', u'an', u'investment', u'for', u'your', u'child', u"'s", u'education', u'or', u'own', u'a', u'mutual', u'fund', u'or', u'a', u'stock', u'in', u'a', u'retirement', u'plan', u',', u'(', u'obama', u')', u'is', u'going', u'to', u'raise', u'your', u'taxes', u'.']
[u'corporate', u'profits', u'are', u'up', u',', u'ceo', u'pay', u'is', u'up', u',', u'but', u'average', u'wages', u'of', u'americans', u'are', u'flat', u'.']
[u'i', u'have', u'35', u'years', u"'", u'experience', u'making', u'change', u'.']
[u'personal', u'income', u'is', u'growing', u'faster', u'in', u'tennessee', u'on', u'average', u'than', u'any', u'other', u'state

In [17]:
data['tokenized_sents'] = data.apply(lambda row: nltk.word_tokenize(row['the statement']), axis=1)

In [18]:
data['the statement'][7]

u"i'm the only person on this stage who has worked actively just last year passing, along with russ feingold, some of the toughest ethics reform since watergate."

In [19]:
data['tokenized_sents']

0        [says, the, annies, list, political, group, su...
1        [when, did, the, decline, of, coal, start, ?, ...
2        [hillary, clinton, agrees, with, john, mccain,...
5        [the, chicago, bears, have, had, more, startin...
7        [i, 'm, the, only, person, on, this, stage, wh...
9        [says, gop, primary, opponents, glenn, grothma...
10       [for, the, first, time, in, history, ,, the, s...
11       [since, 2000, ,, nearly, 12, million, american...
12       [when, mitt, romney, was, governor, of, massac...
14       [most, of, the, (, affordable, care, act, ), h...
15       [in, this, last, election, in, november, ,, .....
16       [mccain, opposed, a, requirement, that, the, g...
18       [water, rates, in, manila, ,, philippines, ,, ...
19       [almost, 100,000, people, left, puerto, rico, ...
20       [women, and, men, both, are, making, less, whe...
22       [we, just, had, the, best, year, for, the, aut...
24       [says, mitt, romney, wants, to, get, rid, of, .

In [24]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized


In [26]:
doc_clean = [clean(doc).split() for doc in data['the statement']]

In [27]:
doc_clean

[[u'say',
  u'annies',
  u'list',
  u'political',
  u'group',
  u'support',
  u'thirdtrimester',
  u'abortion',
  u'demand'],
 [u'decline',
  u'coal',
  u'start',
  u'started',
  u'natural',
  u'gas',
  u'took',
  u'started',
  u'begin',
  u'president',
  u'george',
  u'w',
  u'bush',
  u'administration'],
 [u'hillary',
  u'clinton',
  u'agrees',
  u'john',
  u'mccain',
  u'by',
  u'voting',
  u'give',
  u'george',
  u'bush',
  u'benefit',
  u'doubt',
  u'iran'],
 [u'chicago',
  u'bear',
  u'starting',
  u'quarterback',
  u'last',
  u'10',
  u'year',
  u'total',
  u'number',
  u'tenured',
  u'uw',
  u'faculty',
  u'fired',
  u'last',
  u'two',
  u'decade'],
 [u'im',
  u'person',
  u'stage',
  u'worked',
  u'actively',
  u'last',
  u'year',
  u'passing',
  u'along',
  u'rus',
  u'feingold',
  u'toughest',
  u'ethic',
  u'reform',
  u'since',
  u'watergate'],
 [u'say',
  u'gop',
  u'primary',
  u'opponent',
  u'glenn',
  u'grothman',
  u'joe',
  u'leibham',
  u'cast',
  u'compromise',
  

In [33]:
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [34]:
doc_term_matrix

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 2),
  (20, 1),
  (21, 1)],
 [(11, 1),
  (15, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1)],
 [(33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1)],
 [(39, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(6, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 2),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1)],
 [(76, 2),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 2),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1)],
 [(57, 1

In [35]:
Lda = gensim.models.ldamodel.LdaModel

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [36]:
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [37]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(0, u'0.022*"year" + 0.015*"job" + 0.014*"state"'), (1, u'0.024*"say" + 0.015*"health" + 0.013*"care"'), (2, u'0.021*"percent" + 0.021*"say" + 0.016*"tax"')]


In [50]:
lda_model =  gensim.models.LdaMulticore(doc_term_matrix, 
                                   num_topics = 5, 
                                   id2word = dictionary,                                    
                                   passes = 20,
                                   workers = 2)

In [49]:
lda_model.num_topics
lda_model.print_topics()


[(0,
  u'0.027*"say" + 0.010*"year" + 0.010*"care" + 0.009*"health" + 0.009*"percent" + 0.007*"job" + 0.006*"government" + 0.005*"cost" + 0.005*"million" + 0.005*"tax"'),
 (1,
  u'0.017*"say" + 0.011*"state" + 0.011*"year" + 0.008*"would" + 0.008*"rate" + 0.008*"tax" + 0.007*"percent" + 0.007*"health" + 0.007*"job" + 0.007*"law"'),
 (2,
  u'0.023*"say" + 0.014*"obama" + 0.014*"president" + 0.010*"tax" + 0.009*"year" + 0.009*"country" + 0.009*"barack" + 0.009*"state" + 0.007*"people" + 0.007*"time"'),
 (3,
  u'0.019*"say" + 0.016*"state" + 0.015*"percent" + 0.012*"year" + 0.008*"tax" + 0.008*"job" + 0.007*"school" + 0.005*"u" + 0.005*"wisconsin" + 0.005*"new"'),
 (4,
  u'0.013*"state" + 0.010*"tax" + 0.010*"percent" + 0.009*"say" + 0.007*"american" + 0.007*"year" + 0.007*"people" + 0.007*"texas" + 0.007*"million" + 0.006*"bill"')]

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result



In [ ]:
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test) 

In [ ]:

lda = LDA(n_components=1)  
X_train = lda.fit_transform(X_train, y_train)  
X_test = lda.transform(X_test) 
